In [248]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [249]:
data_url = "train.csv"
data = pd.read_csv(data_url) #pandas로 csv 파일을 불러온다.

In [250]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [251]:
#의미 있는 데이터 
#Pclass : (낮은 등급(1등급)일 수록 생존률 높음
#Sex : 여성일 수록 생존률 높음
#Age : 낮을수록 생존률 높지만. 더미 있음
#Fare : 높을수록 생존률 높음 

input_data = data[["Survived", "Pclass", "Sex", "Age", "Fare"]].dropna() 
#loc등은 행을 불러올 때. 열을 불러올 때는 바로 인덱스 접근하듯이.
#dropna로 NaN 값 삭제
output_data = input_data["Survived"]
input_data = input_data.drop("Survived", axis=1) #축 지정해 줘야 한다. 축 지정하지 않은 drop은 행 삭제
input_data = input_data.replace(["male", "female"], [0.19, 0.74])

x_data = input_data.values
y_data = output_data.values
y_data = np.reshape(y_data, (-1, 1))

In [252]:
X = tf.placeholder(tf.float32, shape=[None, 4], name="input")
Y = tf.placeholder(tf.int32, shape=[None, 1], name="labels")

W = tf.Variable(tf.truncated_normal((4, 2), stddev=0.1), name="weight")
b = tf.Variable(tf.zeros([2]), name="bias")

Y_one_hot = tf.one_hot(Y, 2)
Y_one_hot = tf.reshape(Y_one_hot, [-1, 2])

In [253]:
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)

prediction = tf.argmax(hypothesis, 1) #예측된 값에서 가장 높은 값을 가지는 요소의 인덱스 추출
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) #정답레이블과 예상치 비교
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #정확도 계산

In [262]:
test = pd.read_csv("test.csv")

test_data = test[["Pclass", "Sex", "Age", "Fare"]] 
test_data = test_data.replace(["male", "female", "NaN"], [0.19, 0.74, 30.0])

test_data = test_data.values

print(test_data.shape)

(418, 4)


In [263]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #초기화

    for step in range(55000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data}) #optimizer에 모든 텐서가 연결되어 있으므로 optimizer 실행
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
            
    #학습 이후 테스트
    pred = sess.run(prediction, feed_dict={X: test_data})
    print("TEST")
    for p in pred: #flattend은 1차원으로 변환. [[1], [0]] -> [1, 0]. 
        print(p)


Step:     0	Loss: 0.666	Acc: 61.90%
Step:   100	Loss: 0.639	Acc: 66.67%
Step:   200	Loss: 0.635	Acc: 66.53%
Step:   300	Loss: 0.632	Acc: 65.55%
Step:   400	Loss: 0.629	Acc: 65.55%
Step:   500	Loss: 0.626	Acc: 65.41%
Step:   600	Loss: 0.624	Acc: 65.69%
Step:   700	Loss: 0.622	Acc: 66.25%
Step:   800	Loss: 0.620	Acc: 66.67%
Step:   900	Loss: 0.619	Acc: 66.95%
Step:  1000	Loss: 0.617	Acc: 66.95%
Step:  1100	Loss: 0.616	Acc: 67.09%
Step:  1200	Loss: 0.614	Acc: 67.23%
Step:  1300	Loss: 0.613	Acc: 67.23%
Step:  1400	Loss: 0.612	Acc: 67.93%
Step:  1500	Loss: 0.611	Acc: 67.93%
Step:  1600	Loss: 0.610	Acc: 67.79%
Step:  1700	Loss: 0.609	Acc: 67.93%
Step:  1800	Loss: 0.608	Acc: 67.93%
Step:  1900	Loss: 0.607	Acc: 67.93%
Step:  2000	Loss: 0.606	Acc: 67.93%
Step:  2100	Loss: 0.605	Acc: 68.07%
Step:  2200	Loss: 0.604	Acc: 68.07%
Step:  2300	Loss: 0.603	Acc: 68.21%
Step:  2400	Loss: 0.602	Acc: 68.21%
Step:  2500	Loss: 0.601	Acc: 68.21%
Step:  2600	Loss: 0.600	Acc: 68.21%
Step:  2700	Loss: 0.599	Acc:

Step: 22900	Loss: 0.499	Acc: 78.15%
Step: 23000	Loss: 0.499	Acc: 78.15%
Step: 23100	Loss: 0.498	Acc: 78.15%
Step: 23200	Loss: 0.498	Acc: 78.15%
Step: 23300	Loss: 0.498	Acc: 78.15%
Step: 23400	Loss: 0.498	Acc: 78.15%
Step: 23500	Loss: 0.497	Acc: 78.15%
Step: 23600	Loss: 0.497	Acc: 78.15%
Step: 23700	Loss: 0.497	Acc: 78.15%
Step: 23800	Loss: 0.497	Acc: 78.15%
Step: 23900	Loss: 0.496	Acc: 78.15%
Step: 24000	Loss: 0.496	Acc: 78.15%
Step: 24100	Loss: 0.496	Acc: 78.15%
Step: 24200	Loss: 0.496	Acc: 78.15%
Step: 24300	Loss: 0.495	Acc: 78.15%
Step: 24400	Loss: 0.495	Acc: 78.15%
Step: 24500	Loss: 0.495	Acc: 78.15%
Step: 24600	Loss: 0.495	Acc: 78.15%
Step: 24700	Loss: 0.495	Acc: 78.15%
Step: 24800	Loss: 0.494	Acc: 78.29%
Step: 24900	Loss: 0.494	Acc: 78.43%
Step: 25000	Loss: 0.494	Acc: 78.43%
Step: 25100	Loss: 0.494	Acc: 78.43%
Step: 25200	Loss: 0.493	Acc: 78.43%
Step: 25300	Loss: 0.493	Acc: 78.43%
Step: 25400	Loss: 0.493	Acc: 78.43%
Step: 25500	Loss: 0.493	Acc: 78.43%
Step: 25600	Loss: 0.493	Acc:

Step: 45800	Loss: 0.468	Acc: 79.83%
Step: 45900	Loss: 0.468	Acc: 79.83%
Step: 46000	Loss: 0.467	Acc: 79.69%
Step: 46100	Loss: 0.467	Acc: 79.69%
Step: 46200	Loss: 0.467	Acc: 79.69%
Step: 46300	Loss: 0.467	Acc: 79.69%
Step: 46400	Loss: 0.467	Acc: 79.69%
Step: 46500	Loss: 0.467	Acc: 79.69%
Step: 46600	Loss: 0.467	Acc: 79.69%
Step: 46700	Loss: 0.467	Acc: 79.69%
Step: 46800	Loss: 0.467	Acc: 79.69%
Step: 46900	Loss: 0.467	Acc: 79.69%
Step: 47000	Loss: 0.467	Acc: 79.69%
Step: 47100	Loss: 0.467	Acc: 79.69%
Step: 47200	Loss: 0.467	Acc: 79.69%
Step: 47300	Loss: 0.467	Acc: 79.69%
Step: 47400	Loss: 0.467	Acc: 79.69%
Step: 47500	Loss: 0.466	Acc: 79.69%
Step: 47600	Loss: 0.466	Acc: 79.69%
Step: 47700	Loss: 0.466	Acc: 79.69%
Step: 47800	Loss: 0.466	Acc: 79.69%
Step: 47900	Loss: 0.466	Acc: 79.69%
Step: 48000	Loss: 0.466	Acc: 79.69%
Step: 48100	Loss: 0.466	Acc: 79.69%
Step: 48200	Loss: 0.466	Acc: 79.69%
Step: 48300	Loss: 0.466	Acc: 79.69%
Step: 48400	Loss: 0.466	Acc: 79.69%
Step: 48500	Loss: 0.466	Acc: